In [5]:
# General import
import time
import math
import numpy as np
import pandas as pd
from scipy import linalg

# local functions import
from functions import  *

ImportError: attempted relative import with no known parent package

In [1]:

split_data_tp1_app = open_file('../data/data_tp1_app.txt')
split_data_tp1_dec = open_file('../data/data_tp1_dec.txt')

split_data_tp2_app = open_file('../data/data_tp2_app.txt')
split_data_tp2_dec = open_file('../data/data_tp2_dec.txt')

split_data_tp3_app = open_file('../data/data_tp3_app.txt')
split_data_tp3_dec = open_file('../data/data_tp3_dec.txt')

NameError: name 'open_file' is not defined

In [3]:
def gaussian_fit_model(data):
    class_centers = {}
    classes = get_unique_class_num(data)
    for class_num in classes:
        data_class = get_splited_class(data, int(class_num))
        sum_x = 0
        sum_y = 0
        for line in data_class:
            sum_x += float(line[1])
            sum_y += float(line[2])
        x_center = 1/len(data_class) * sum_x
        y_center = 1/len(data_class) * sum_y
        class_centers[class_num] = [x_center, y_center]
    return class_centers

def compute_one_euclidian_dist(first_point, second_point):
    sum_square = 0
    for i in range(0, len(first_point)):
        sum_square = (float(first_point[i]) - float(second_point[i]))**2      
    return math.sqrt(sum_square)

def compute_euclidian_dists(class_centers, line):
    dists_dict = {}
    for class_center in class_centers:
        dist = compute_one_euclidian_dist(class_centers[class_center], line[1:3])
        dists_dict[class_center] = dist   
    return dists_dict

def euclidian_test_model(class_centers, app_data, dec_data):
    classes_num =get_unique_class_num(app_data)
    conf_matrix = np.zeros((len(classes_num), len(classes_num)))
    count_top1 = 0
    count_top2 = 0
    for line in dec_data:
        dists = compute_euclidian_dists(class_centers, line)
        if get_top_n_decision(1, line[0], dists):
            count_top1 = count_top1 + 1
        if get_top_n_decision(2, line[0], dists):
            count_top2 = count_top2 + 1
        update_confusion_matrix(conf_matrix, line[0], dists)
    df = transform_matrix_to_df(conf_matrix, classes_num)
    print_decision_model_result(len(app_data), len(dec_data), count_top1/len(dec_data), count_top2/len(dec_data), df)


In [4]:
def compute_mahalanobis_dists(class_centers, line, inv_covmat):
    dists_dict = {}
    for class_center in class_centers:
        dist = compute_one_mahalanobis_dist(line[1:3], class_centers[class_center], inv_covmat)
        dists_dict[class_center] = dist   
    return dists_dict

def compute_inv_covmat(dec_data):
    data  = pd.DataFrame(dec_data, dtype=float)[[1,2]]
    cov = np.cov(data.values.T)
    inv_covmat = linalg.inv(cov)
    return inv_covmat

def compute_one_mahalanobis_dist(first_point, second_point, inv_covmat):
    x_minus_mu = []
    for i in range(0, len(first_point)):
        x_minus_mu.append(float(first_point[i]) - second_point[i])
    x_minus_mu = np.array(x_minus_mu)
    left_term = np.dot(x_minus_mu, inv_covmat)
    mahal = np.dot(left_term, x_minus_mu.T)
    return math.sqrt(mahal)

def mahalanobis_test_model(class_centers, app_data, dec_data):
    classes_num =get_unique_class_num(app_data)
    conf_matrix = np.zeros((len(classes_num), len(classes_num)))
    count_top1 = 0
    count_top2 = 0
    inv_covmat = compute_inv_covmat(dec_data)
    for line in dec_data:
        dists = compute_mahalanobis_dists(class_centers, line, inv_covmat)
        if get_top_n_decision(1, line[0], dists):
            count_top1 = count_top1 + 1
        if get_top_n_decision(2, line[0], dists):
            count_top2 = count_top2 + 1
        update_confusion_matrix(conf_matrix, line[0], dists)
    df = transform_matrix_to_df(conf_matrix, classes_num)
    print_decision_model_result(len(app_data), len(dec_data), count_top1/len(dec_data), count_top2/len(dec_data), df)



In [5]:
print("First Data TP1")
start_time = time.time()
model = gaussian_fit_model(split_data_tp1_app)
euclidian_test_model(model, split_data_tp1_app, split_data_tp1_dec)
print("\tExecution time : %s seconds\n" % (time.time() - start_time))
print("==========================\n")

print("First Data TP2")
start_time = time.time()
model = gaussian_fit_model(split_data_tp2_app)
euclidian_test_model(model, split_data_tp2_app, split_data_tp2_dec)
print("\tExecution time : %s seconds\n" % (time.time() - start_time))
print("==========================\n")

print("First Data TP3")
start_time = time.time()
model = gaussian_fit_model(split_data_tp3_app)
euclidian_test_model(model, split_data_tp3_app, split_data_tp3_dec)
print("\tExecution time : %s seconds\n" % (time.time() - start_time))
print("==========================\n")

First Data TP1
	Results :
	----------------
	Number of elements for the learning step :  500
	Number of elements for the decision step :  500
	----------------

	Top results :
	----------------
	Top 1 rate :  0.576
	Top 2 rate :  0.962
	----------------

	Confusion matrix :
	----------------
╒════╤═════╤═════╤═════╤═════╤═════╕
│    │   1 │   2 │   3 │   4 │   5 │
╞════╪═════╪═════╪═════╪═════╪═════╡
│  1 │  40 │  56 │   0 │   0 │   4 │
├────┼─────┼─────┼─────┼─────┼─────┤
│  2 │  41 │  51 │   0 │   0 │   8 │
├────┼─────┼─────┼─────┼─────┼─────┤
│  3 │   0 │   0 │  42 │  58 │   0 │
├────┼─────┼─────┼─────┼─────┼─────┤
│  4 │   0 │   0 │  34 │  60 │   6 │
├────┼─────┼─────┼─────┼─────┼─────┤
│  5 │   1 │   0 │   4 │   0 │  95 │
╘════╧═════╧═════╧═════╧═════╧═════╛
	----------------
	Execution time : 0.012201547622680664 seconds


First Data TP2
	Results :
	----------------
	Number of elements for the learning step :  500
	Number of elements for the decision step :  500
	----------------

In [6]:
print("First Data TP1")
start_time = time.time()
model = gaussian_fit_model(split_data_tp1_app)
mahalanobis_test_model(model, split_data_tp1_app, split_data_tp1_dec)
print("\tExecution time : %s seconds\n" % (time.time() - start_time))
print("==========================\n")

print("First Data TP2")
start_time = time.time()
model = gaussian_fit_model(split_data_tp2_app)
mahalanobis_test_model(model, split_data_tp2_app, split_data_tp2_dec)
print("\tExecution time : %s seconds\n" % (time.time() - start_time))
print("==========================\n")

print("First Data TP3")
start_time = time.time()
model = gaussian_fit_model(split_data_tp3_app)
mahalanobis_test_model(model, split_data_tp3_app, split_data_tp3_dec)
print("\tExecution time : %s seconds\n" % (time.time() - start_time))
print("==========================\n")

First Data TP1
	Results :
	----------------
	Number of elements for the learning step :  500
	Number of elements for the decision step :  500
	----------------

	Top results :
	----------------
	Top 1 rate :  0.992
	Top 2 rate :  1.0
	----------------

	Confusion matrix :
	----------------
╒════╤═════╤═════╤═════╤═════╤═════╕
│    │   1 │   2 │   3 │   4 │   5 │
╞════╪═════╪═════╪═════╪═════╪═════╡
│  1 │  98 │   0 │   0 │   0 │   2 │
├────┼─────┼─────┼─────┼─────┼─────┤
│  2 │   0 │ 100 │   0 │   0 │   0 │
├────┼─────┼─────┼─────┼─────┼─────┤
│  3 │   0 │   0 │  99 │   1 │   0 │
├────┼─────┼─────┼─────┼─────┼─────┤
│  4 │   0 │   0 │   0 │ 100 │   0 │
├────┼─────┼─────┼─────┼─────┼─────┤
│  5 │   1 │   0 │   0 │   0 │  99 │
╘════╧═════╧═════╧═════╧═════╧═════╛
	----------------
	Execution time : 0.05135011672973633 seconds


First Data TP2
	Results :
	----------------
	Number of elements for the learning step :  500
	Number of elements for the decision step :  500
	----------------

	